# SQLi World Model

In [1]:
import os, datetime
import numpy as np
import joblib
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras.layers import Input, LSTM, Dense, Embedding
from tensorflow.keras.preprocessing.text import Tokenizer,text_to_word_sequence

## Collect HTML files

Load the HTML files

In [2]:
files = []

for f in os.listdir(os.path.join(os.getcwd(),'html')):
    fd = open(os.path.join(os.getcwd(),'html',f),'r')
    content = fd.read()
    files.append(content)
    fd.close()

Show a sample file

In [3]:
files[0]

'\n<!-- saved from url=(0042)http://jabba.hackingarena.no:806/index.php -->\n<html><head><meta http-equiv="Content-Type" content="text/html; charset=UTF-8"></head><body>Login as admin for the flag!\n<br>some content, some content\n<form action="http://jabba.hackingarena.no:806/index.php" method="post">\n<table width="100">\n<tbody><tr><td>Name:</td>\n<td><input type="text" name="username" value=""></td></tr>\n<tr><td>Password:</td>\n<td><input type="text" name="passwd" value=""></td></tr>\n\n<tr><td><input type="submit" value="Submit"></td></tr>\n</tbody></table>\n</form>\n\n</body></html>\n'

Add a mock file as a concatentation of a previous one

In [4]:
files.append(files[0]*10)

## Tokenize the files

In [5]:
lower = False
filters = '"#$%&()*+,.;?@[\\]^_`{|}~\t\n'

padding='post'

maxlen=200
truncating='post'

Show an example of tokenization

In [6]:
text_to_word_sequence(files[0],lower=lower, filters=filters)

['<!--',
 'saved',
 'from',
 'url=',
 '0042',
 'http://jabba',
 'hackingarena',
 'no:806/index',
 'php',
 '-->',
 '<html><head><meta',
 'http-equiv=',
 'Content-Type',
 'content=',
 'text/html',
 'charset=UTF-8',
 '></head><body>Login',
 'as',
 'admin',
 'for',
 'the',
 'flag!',
 '<br>some',
 'content',
 'some',
 'content',
 '<form',
 'action=',
 'http://jabba',
 'hackingarena',
 'no:806/index',
 'php',
 'method=',
 'post',
 '>',
 '<table',
 'width=',
 '100',
 '>',
 '<tbody><tr><td>Name:</td>',
 '<td><input',
 'type=',
 'text',
 'name=',
 'username',
 'value=',
 '></td></tr>',
 '<tr><td>Password:</td>',
 '<td><input',
 'type=',
 'text',
 'name=',
 'passwd',
 'value=',
 '></td></tr>',
 '<tr><td><input',
 'type=',
 'submit',
 'value=',
 'Submit',
 '></td></tr>',
 '</tbody></table>',
 '</form>',
 '</body></html>']

Learn a tokenization dictionary and convert the pages into token sequences:

In [8]:
SQLi_tokenizer = Tokenizer(lower=lower, filters=filters)
SQLi_tokenizer.fit_on_texts(files)
tokens_list = SQLi_tokenizer.texts_to_sequences(files)

Pad and trim the sequences to fit them into an array:

In [9]:
tokens_array = keras.preprocessing.sequence.pad_sequences(tokens_list, padding=padding, maxlen=maxlen, truncating=truncating)

## Generating a target vector for teacher forcing

We set up a shifted (by -1) version of the input to allow the model to learn a step ahead.

In [11]:
target_array = np.zeros_like(tokens_array)
target_array[:,1:] = tokens_array[:,0:-1]

## Converting into one-hot encoding

Translating integer encoding into one-hot.

In [12]:
tokens_1hot_array = keras.utils.to_categorical(tokens_array)
target_1hot_array = keras.utils.to_categorical(target_array)

In [13]:
tokens_1hot_array.shape

(5, 200, 55)

## Defining Seq2Seq

Instantiating a VAE module built following [https://blog.keras.io/a-ten-minute-introduction-to-sequence-to-sequence-learning-in-keras.html](https://blog.keras.io/a-ten-minute-introduction-to-sequence-to-sequence-learning-in-keras.html).

In [14]:
latent_dim = 3
num_tokens = tokens_1hot_array.shape[2]

encoder_inputs = Input(shape=(None, num_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outpus, state_h, state_c = encoder(encoder_inputs)
encoder_states = [state_h, state_c]

decoder_inputs = Input(shape=(None,num_tokens))
decoder = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs,_,_ = decoder(decoder_inputs, initial_state=encoder_states)
decoder_dense = Dense(num_tokens,activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

model = keras.Model([encoder_inputs,decoder_inputs],decoder_outputs)
model.summary()
    

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None, 55)]   0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, None, 55)]   0                                            
__________________________________________________________________________________________________
lstm (LSTM)                     [(None, 3), (None, 3 708         input_1[0][0]                    
__________________________________________________________________________________________________
lstm_1 (LSTM)                   [(None, None, 3), (N 708         input_2[0][0]                    
                                                                 lstm[0][1]            

In [15]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')

In [16]:
model.fit([tokens_1hot_array,tokens_1hot_array],target_1hot_array,batch_size=5,epochs=2000)

Epoch 1/2000
1/1 [==============================] - 0s 870us/step - loss: 4.0577
Epoch 2/2000
1/1 [==============================] - 0s 704us/step - loss: 4.0483
Epoch 3/2000
1/1 [==============================] - 0s 2ms/step - loss: 4.0418
Epoch 4/2000
1/1 [==============================] - 0s 903us/step - loss: 4.0365
Epoch 5/2000
1/1 [==============================] - 0s 707us/step - loss: 4.0319
Epoch 6/2000
1/1 [==============================] - 0s 721us/step - loss: 4.0277
Epoch 7/2000
1/1 [==============================] - 0s 4ms/step - loss: 4.0238
Epoch 8/2000
1/1 [==============================] - 0s 668us/step - loss: 4.0200
Epoch 9/2000
1/1 [==============================] - 0s 819us/step - loss: 4.0164
Epoch 10/2000
1/1 [==============================] - 0s 668us/step - loss: 4.0130
Epoch 11/2000
1/1 [==============================] - 0s 682us/step - loss: 4.0096
Epoch 12/2000
1/1 [==============================] - 0s 2ms/step - loss: 4.0063
Epoch 13/2000
1/1 [============

1/1 [==============================] - 0s 4ms/step - loss: 3.7017
Epoch 103/2000
1/1 [==============================] - 0s 2ms/step - loss: 3.6975
Epoch 104/2000
1/1 [==============================] - 0s 1ms/step - loss: 3.6932
Epoch 105/2000
1/1 [==============================] - 0s 1ms/step - loss: 3.6889
Epoch 106/2000
1/1 [==============================] - 0s 4ms/step - loss: 3.6847
Epoch 107/2000
1/1 [==============================] - 0s 1ms/step - loss: 3.6804
Epoch 108/2000
1/1 [==============================] - 0s 1ms/step - loss: 3.6762
Epoch 109/2000
1/1 [==============================] - 0s 1ms/step - loss: 3.6719
Epoch 110/2000
1/1 [==============================] - 0s 1ms/step - loss: 3.6677
Epoch 111/2000
1/1 [==============================] - 0s 2ms/step - loss: 3.6634
Epoch 112/2000
1/1 [==============================] - 0s 1ms/step - loss: 3.6592
Epoch 113/2000
1/1 [==============================] - 0s 1ms/step - loss: 3.6549
Epoch 114/2000
1/1 [=======================

1/1 [==============================] - 0s 1ms/step - loss: 3.2704
Epoch 204/2000
1/1 [==============================] - 0s 1ms/step - loss: 3.2661
Epoch 205/2000
1/1 [==============================] - 0s 1ms/step - loss: 3.2619
Epoch 206/2000
1/1 [==============================] - 0s 2ms/step - loss: 3.2576
Epoch 207/2000
1/1 [==============================] - 0s 2ms/step - loss: 3.2533
Epoch 208/2000
1/1 [==============================] - 0s 4ms/step - loss: 3.2490
Epoch 209/2000
1/1 [==============================] - 0s 982us/step - loss: 3.2448
Epoch 210/2000
1/1 [==============================] - 0s 746us/step - loss: 3.2405
Epoch 211/2000
1/1 [==============================] - 0s 756us/step - loss: 3.2363
Epoch 212/2000
1/1 [==============================] - 0s 1ms/step - loss: 3.2320
Epoch 213/2000
1/1 [==============================] - 0s 1ms/step - loss: 3.2278
Epoch 214/2000
1/1 [==============================] - 0s 1ms/step - loss: 3.2235
Epoch 215/2000
1/1 [=================

1/1 [==============================] - 0s 747us/step - loss: 2.8562
Epoch 304/2000
1/1 [==============================] - 0s 712us/step - loss: 2.8521
Epoch 305/2000
1/1 [==============================] - 0s 886us/step - loss: 2.8480
Epoch 306/2000
1/1 [==============================] - 0s 744us/step - loss: 2.8440
Epoch 307/2000
1/1 [==============================] - 0s 1ms/step - loss: 2.8399
Epoch 308/2000
1/1 [==============================] - 0s 2ms/step - loss: 2.8359
Epoch 309/2000
1/1 [==============================] - 0s 1ms/step - loss: 2.8318
Epoch 310/2000
1/1 [==============================] - 0s 6ms/step - loss: 2.8278
Epoch 311/2000
1/1 [==============================] - 0s 2ms/step - loss: 2.8237
Epoch 312/2000
1/1 [==============================] - 0s 1ms/step - loss: 2.8197
Epoch 313/2000
1/1 [==============================] - 0s 813us/step - loss: 2.8156
Epoch 314/2000
1/1 [==============================] - 0s 946us/step - loss: 2.8116
Epoch 315/2000
1/1 [===========

1/1 [==============================] - 0s 755us/step - loss: 2.4670
Epoch 404/2000
1/1 [==============================] - 0s 741us/step - loss: 2.4633
Epoch 405/2000
1/1 [==============================] - 0s 1ms/step - loss: 2.4597
Epoch 406/2000
1/1 [==============================] - 0s 3ms/step - loss: 2.4561
Epoch 407/2000
1/1 [==============================] - 0s 2ms/step - loss: 2.4525
Epoch 408/2000
1/1 [==============================] - 0s 1ms/step - loss: 2.4490
Epoch 409/2000
1/1 [==============================] - 0s 1ms/step - loss: 2.4454
Epoch 410/2000
1/1 [==============================] - 0s 1ms/step - loss: 2.4418
Epoch 411/2000
1/1 [==============================] - 0s 2ms/step - loss: 2.4382
Epoch 412/2000
1/1 [==============================] - 0s 1ms/step - loss: 2.4347
Epoch 413/2000
1/1 [==============================] - 0s 1ms/step - loss: 2.4311
Epoch 414/2000
1/1 [==============================] - 0s 1ms/step - loss: 2.4276
Epoch 415/2000
1/1 [===================

...

1/1 [==============================] - 0s 1ms/step - loss: 1.1323
Epoch 1598/2000
1/1 [==============================] - 0s 1ms/step - loss: 1.1319
Epoch 1599/2000
1/1 [==============================] - 0s 766us/step - loss: 1.1314
Epoch 1600/2000
1/1 [==============================] - 0s 1ms/step - loss: 1.1310
Epoch 1601/2000
1/1 [==============================] - 0s 757us/step - loss: 1.1306
Epoch 1602/2000
1/1 [==============================] - 0s 775us/step - loss: 1.1301
Epoch 1603/2000
1/1 [==============================] - 0s 781us/step - loss: 1.1297
Epoch 1604/2000
1/1 [==============================] - 0s 1ms/step - loss: 1.1293
Epoch 1605/2000
1/1 [==============================] - 0s 836us/step - loss: 1.1288
Epoch 1606/2000
1/1 [==============================] - 0s 802us/step - loss: 1.1284
Epoch 1607/2000
1/1 [==============================] - 0s 9ms/step - loss: 1.1280
Epoch 1608/2000
1/1 [==============================] - 0s 6ms/step - loss: 1.1275
Epoch 1609/2000
1/1 

1/1 [==============================] - 0s 1ms/step - loss: 1.0910
Epoch 1697/2000
1/1 [==============================] - 0s 1ms/step - loss: 1.0906
Epoch 1698/2000
1/1 [==============================] - 0s 1ms/step - loss: 1.0902
Epoch 1699/2000
1/1 [==============================] - 0s 3ms/step - loss: 1.0898
Epoch 1700/2000
1/1 [==============================] - 0s 6ms/step - loss: 1.0895
Epoch 1701/2000
1/1 [==============================] - 0s 913us/step - loss: 1.0891
Epoch 1702/2000
1/1 [==============================] - 0s 2ms/step - loss: 1.0887
Epoch 1703/2000
1/1 [==============================] - 0s 2ms/step - loss: 1.0883
Epoch 1704/2000
1/1 [==============================] - 0s 1ms/step - loss: 1.0879
Epoch 1705/2000
1/1 [==============================] - 0s 2ms/step - loss: 1.0875
Epoch 1706/2000
1/1 [==============================] - 0s 2ms/step - loss: 1.0871
Epoch 1707/2000
1/1 [==============================] - 0s 1ms/step - loss: 1.0867
Epoch 1708/2000
1/1 [=========

1/1 [==============================] - 0s 1ms/step - loss: 1.0537
Epoch 1796/2000
1/1 [==============================] - 0s 1ms/step - loss: 1.0534
Epoch 1797/2000
1/1 [==============================] - 0s 4ms/step - loss: 1.0530
Epoch 1798/2000
1/1 [==============================] - 0s 1ms/step - loss: 1.0526
Epoch 1799/2000
1/1 [==============================] - 0s 1ms/step - loss: 1.0523
Epoch 1800/2000
1/1 [==============================] - 0s 2ms/step - loss: 1.0519
Epoch 1801/2000
1/1 [==============================] - 0s 893us/step - loss: 1.0516
Epoch 1802/2000
1/1 [==============================] - 0s 1ms/step - loss: 1.0512
Epoch 1803/2000
1/1 [==============================] - 0s 1ms/step - loss: 1.0509
Epoch 1804/2000
1/1 [==============================] - 0s 2ms/step - loss: 1.0505
Epoch 1805/2000
1/1 [==============================] - 0s 3ms/step - loss: 1.0502
Epoch 1806/2000
1/1 [==============================] - 0s 1ms/step - loss: 1.0498
Epoch 1807/2000
1/1 [=========

1/1 [==============================] - 0s 755us/step - loss: 1.0202
Epoch 1895/2000
1/1 [==============================] - 0s 750us/step - loss: 1.0199
Epoch 1896/2000
1/1 [==============================] - 0s 768us/step - loss: 1.0195
Epoch 1897/2000
1/1 [==============================] - 0s 762us/step - loss: 1.0192
Epoch 1898/2000
1/1 [==============================] - 0s 803us/step - loss: 1.0189
Epoch 1899/2000
1/1 [==============================] - 0s 757us/step - loss: 1.0186
Epoch 1900/2000
1/1 [==============================] - 0s 869us/step - loss: 1.0183
Epoch 1901/2000
1/1 [==============================] - 0s 1ms/step - loss: 1.0179
Epoch 1902/2000
1/1 [==============================] - 0s 1ms/step - loss: 1.0176
Epoch 1903/2000
1/1 [==============================] - 0s 4ms/step - loss: 1.0173
Epoch 1904/2000
1/1 [==============================] - 0s 953us/step - loss: 1.0170
Epoch 1905/2000
1/1 [==============================] - 0s 826us/step - loss: 1.0167
Epoch 1906/200

1/1 [==============================] - 0s 2ms/step - loss: 0.9902
Epoch 1994/2000
1/1 [==============================] - 0s 1ms/step - loss: 0.9899
Epoch 1995/2000
1/1 [==============================] - 0s 795us/step - loss: 0.9896
Epoch 1996/2000
1/1 [==============================] - 0s 743us/step - loss: 0.9893
Epoch 1997/2000
1/1 [==============================] - 0s 1ms/step - loss: 0.9890
Epoch 1998/2000
1/1 [==============================] - 0s 1ms/step - loss: 0.9888
Epoch 1999/2000
1/1 [==============================] - 0s 1ms/step - loss: 0.9885
Epoch 2000/2000
1/1 [==============================] - 0s 1ms/step - loss: 0.9882


## Checking the Seq2Seq

In [17]:
encoder_model = keras.Model(encoder_inputs,encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_input = [decoder_state_input_h,decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder(decoder_inputs, initial_state=decoder_states_input)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = keras.Model([decoder_inputs]+decoder_states_input,
                           [decoder_outputs]+decoder_states)

In [20]:
def decode_sequence(input_seq):
    states_value = encoder_model.predict(input_seq)
    print(states_value)

    target_seq = np.zeros((1, 1, num_tokens))
    target_seq[0, 0, 0] = 1.

    decoded_sentence = []
    for _ in range(maxlen):
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        decoded_sentence.append(sampled_token_index)

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence

In [21]:
decode_sequence(tokens_1hot_array[1:2,:,:])

[array([[0.9751636 , 0.26560563, 0.98514694]], dtype=float32), array([[2.3536913 , 0.65040624, 2.7095926 ]], dtype=float32)]


[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0]

In [22]:
decode_sequence(tokens_1hot_array[3:4,:,:])

[array([[0.9751636 , 0.26560563, 0.98514694]], dtype=float32), array([[2.3536913 , 0.65040624, 2.7095926 ]], dtype=float32)]


[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0]

latent_dim = 3


encoder_inputs = Input(shape=(None, tokens_per_file))
x = Embedding(tokens_per_file,latent_dim)(encoder_inputs)
x, state_h, state_c = LSTM(latent_dim, return_state=True)(x)
encoder_states = [state_h, state_c]

decoder_inputs = Input(shape=(tokens_per_file,))
x = Embedding(tokens_per_file,latent_dim)(decoder_inputs)
x = LSTM(latent_dim, return_sequences=True)(x, initial_state=encoder_states)
decoder_outputs = Dense(tokens_per_file,activation='softmax')(x)

model = keras.Model([encoder_inputs,decoder_inputs],decoder_outputs)
model.summary()
    

## Checking the VAE

Printing the encoding of the first three data points (showing $\mu_\phi, \log\sigma_\phi, z$)

In [66]:
vae.encoder(tokens_array[0:3])

[<tf.Tensor: shape=(3, 2), dtype=float32, numpy=
 array([[ 2.3456044 , -1.4626523 ],
        [-2.4570012 ,  2.0801358 ],
        [-0.23118107,  4.569852  ]], dtype=float32)>,
 <tf.Tensor: shape=(3, 2), dtype=float32, numpy=
 array([[-4.406884 , -4.8628297],
        [-1.7679614, -2.3994868],
        [-1.6476262, -2.249241 ]], dtype=float32)>,
 <tf.Tensor: shape=(3, 2), dtype=float32, numpy=
 array([[ 2.2758093, -1.4036723],
        [-2.2363672,  2.0302722],
        [-0.3137067,  4.6862164]], dtype=float32)>]

Computing the encoding and decoding of the first datapoint

In [67]:
xtilde = vae.decoder(vae.encoder(tokens_array[0:1]))

Rounding the computed representation to integers

In [68]:
xtilde_int = np.round(xtilde)

Printing the original and the reconstructed representation.

In [69]:
len(tokens_array[0])

200

In [70]:
tokens_array[0]

array([13, 14, 15, 16, 47,  4,  5, 10, 11, 17, 18, 19, 20, 21, 22, 23, 24,
       25, 26, 27, 28, 29, 48, 12, 49, 12, 30, 31,  4,  5, 10, 11, 32, 33,
        6, 34, 35, 36,  6, 37,  7,  1,  8,  9, 38,  2,  3, 39,  7,  1,  8,
        9, 40,  2,  3, 41,  1, 42,  2, 43,  3, 44, 45, 46, 55, 55, 55, 55,
       55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55,
       55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55,
       55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55,
       55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55,
       55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55,
       55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55,
       55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55,
       55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55])

In [71]:
xtilde_int.shape

(1, 200)

In [72]:
xtilde_int

array([[13., 14., 15., 16., 47.,  0.,  5.,  9., 11.,  0., 18., 19., 20.,
        21., 22., 23., 24., 25., 26., 27., 28., 29.,  0., 21., 46., 15.,
        24., 25.,  5.,  8., 14., 14., 27., 34., 12., 34., 30., 35.,  6.,
        28.,  7.,  0., 13.,  8., 32.,  0.,  3., 30.,  7.,  0., 13.,  8.,
        32.,  0.,  0.,  0.,  0., 40.,  0., 42.,  8., 44., 46., 47., 55.,
        54., 54., 55., 55., 55., 55., 55., 55., 55., 55.,  0., 55., 55.,
        55., 55., 55., 55., 55., 55., 55., 55., 55.,  0., 55., 55., 55.,
        55., 55., 55., 55., 55., 55., 55., 55., 55., 55., 55., 55., 55.,
        55., 55., 55., 55., 55., 55., 55., 55., 55., 55., 55., 55.,  0.,
        55., 55., 55., 55., 54., 55., 55., 55., 55., 55., 55., 55., 55.,
        55., 55., 55., 55.,  0., 55., 55., 54., 55., 55., 55., 55., 55.,
         0., 55., 55., 55., 55., 55., 55., 54.,  0., 55., 55., 55., 55.,
        55., 55., 55., 55., 55., 55., 55., 54., 55., 55., 55., 55., 55.,
        55., 55., 55., 54., 55., 55., 55., 55., 55.

In [73]:
tknizer.sequences_to_texts(tokens_array[0:1])

['<!-- saved from url= 0042 http://jabba hackingarena no:806/index php --> <html><head><meta http-equiv= Content-Type content= text/html charset=UTF-8 ></head><body>Login as admin for the flag! <br>some content some content <form action= http://jabba hackingarena no:806/index php method= post > <table width= 100 > <tbody><tr><td>Name:</td> <td><input type= text name= username value= ></td></tr> <tr><td>Password:</td> <td><input type= text name= passwd value= ></td></tr> <tr><td><input type= submit value= Submit ></td></tr> </tbody></table> </form> </body></html>']

In [74]:
tknizer.sequences_to_texts(xtilde_int)

['<!-- saved from url= 0042 hackingarena name= php <html><head><meta http-equiv= Content-Type content= text/html charset=UTF-8 ></head><body>Login as admin for the flag! content= </body></html> from ></head><body>Login as hackingarena text saved saved for <table content <table <form width= > the <td><input <!-- text method= ></td></tr> <form <td><input <!-- text method= passwd submit text </tbody></table> </body></html> 0042 login login login login login login login']

In [75]:
np.mean(xtilde_int==tokens_array[0])

0.735

## Saving/loading the VAE

Saving the model weights

In [29]:
timestamp = datetime.datetime.now().strftime("%Y%m%d%H%M%S%f")
vae.save_weights('ignore_vae_W_'+timestamp)

Loading the model weights

In [30]:
vae1 = VAE(encoder,decoder)
vae1.load_weights('ignore_vae_W_20210228141045773142')

Checking the weights

In [31]:
vae.decoder(vae.encoder(tokens_array[0:1])) == vae1.decoder(vae.encoder(tokens_array[0:1]))

<tf.Tensor: shape=(1, 200), dtype=bool, numpy=
array([[ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  

## Define a mock server

We import and instantiate a new server.

In [32]:
import SQLiServer

In [33]:
env = SQLiServer.mockSQLienv(files)

In [34]:
env.reset()

('\n<!-- saved from url=(0033)http://jabba.hackingarena.no:806/ -->\n<html><head><meta http-equiv="Content-Type" content="text/html; charset=UTF-8"></head><body>Login as admin for the flag!\n\n<form action="http://jabba.hackingarena.no:806/index.php" method="post">\n<table width="100">\n<tbody><tr><td>Name:</td>\n<td><input type="text" name="username" value=""></td></tr>\n<tr><td>Password:</td>\n<td><input type="text" name="passwd" value=""></td></tr>\n\n<tr><td><input type="submit" value="Submit"></td></tr>\n</tbody></table>\n</form>\n\n</body></html>',
 0,
 False,
 'env reset')

We check the current solution.

In [43]:
S0,S1 = env._get_solution()

We take a couple of random steps (corresponding to *wrong request*)

In [37]:
env.step(2)

('\n<!-- saved from url=(0042)http://jabba.hackingarena.no:806/index.php -->\n<html><head><meta http-equiv="Content-Type" content="text/html; charset=UTF-8"></head><body>Login as admin for the flag!\n<br>incorrect login\n<form action="http://jabba.hackingarena.no:806/index.php" method="post">\n<table width="100">\n<tbody><tr><td>Name:</td>\n<td><input type="text" name="username" value=""></td></tr>\n<tr><td>Password:</td>\n<td><input type="text" name="passwd" value=""></td></tr>\n\n<tr><td><input type="submit" value="Submit"></td></tr>\n</tbody></table>\n</form>\n\n</body></html>',
 -1,
 False,
 'wrong request')

In [38]:
env.step(S1)

('\n<!-- saved from url=(0042)http://jabba.hackingarena.no:806/index.php -->\n<html><head><meta http-equiv="Content-Type" content="text/html; charset=UTF-8"></head><body>Login as admin for the flag!\n<br>incorrect login\n<form action="http://jabba.hackingarena.no:806/index.php" method="post">\n<table width="100">\n<tbody><tr><td>Name:</td>\n<td><input type="text" name="username" value=""></td></tr>\n<tr><td>Password:</td>\n<td><input type="text" name="passwd" value=""></td></tr>\n\n<tr><td><input type="submit" value="Submit"></td></tr>\n</tbody></table>\n</form>\n\n</body></html>',
 -1,
 False,
 'wrong request')

We discover the right escape character.

In [39]:
env.step(S0)

('\n<!-- saved from url=(0042)http://jabba.hackingarena.no:806/index.php -->\n<html><head><meta http-equiv="Content-Type" content="text/html; charset=UTF-8"></head><body>Login as admin for the flag!\n<br>some content, some content\n<form action="http://jabba.hackingarena.no:806/index.php" method="post">\n<table width="100">\n<tbody><tr><td>Name:</td>\n<td><input type="text" name="username" value=""></td></tr>\n<tr><td>Password:</td>\n<td><input type="text" name="passwd" value=""></td></tr>\n\n<tr><td><input type="submit" value="Submit"></td></tr>\n</tbody></table>\n</form>\n\n</body></html>\n',
 -1,
 False,
 'correct escape')

We take another couple of random steps.

In [40]:
env.step(2)

('\n<!-- saved from url=(0042)http://jabba.hackingarena.no:806/index.php -->\n<html><head><meta http-equiv="Content-Type" content="text/html; charset=UTF-8"></head><body>Login as admin for the flag!\n<br>incorrect login\n<form action="http://jabba.hackingarena.no:806/index.php" method="post">\n<table width="100">\n<tbody><tr><td>Name:</td>\n<td><input type="text" name="username" value=""></td></tr>\n<tr><td>Password:</td>\n<td><input type="text" name="passwd" value=""></td></tr>\n\n<tr><td><input type="submit" value="Submit"></td></tr>\n</tbody></table>\n</form>\n\n</body></html>',
 -1,
 False,
 'wrong request')

In [41]:
env.step(S0)

('\n<!-- saved from url=(0042)http://jabba.hackingarena.no:806/index.php -->\n<html><head><meta http-equiv="Content-Type" content="text/html; charset=UTF-8"></head><body>Login as admin for the flag!\n<br>incorrect login\n<form action="http://jabba.hackingarena.no:806/index.php" method="post">\n<table width="100">\n<tbody><tr><td>Name:</td>\n<td><input type="text" name="username" value=""></td></tr>\n<tr><td>Password:</td>\n<td><input type="text" name="passwd" value=""></td></tr>\n\n<tr><td><input type="submit" value="Submit"></td></tr>\n</tbody></table>\n</form>\n\n</body></html>',
 -1,
 False,
 'wrong request')

We find the flag

In [42]:
env.step(S1)

('\n<!-- saved from url=(0042)http://jabba.hackingarena.no:806/index.php -->\n<html><head><meta http-equiv="Content-Type" content="text/html; charset=UTF-8"></head><body>Login as admin for the flag!\n<br>flag\n<form action="http://jabba.hackingarena.no:806/index.php" method="post">\n<table width="100">\n<tbody><tr><td>Name:</td>\n<td><input type="text" name="username" value=""></td></tr>\n<tr><td>Password:</td>\n<td><input type="text" name="passwd" value=""></td></tr>\n\n<tr><td><input type="submit" value="Submit"></td></tr>\n</tbody></table>\n</form>\n\n</body></html>\n',
 100,
 True,
 'flag')